1. Databases Creation Using Python

In [33]:

import sqlite3
import pandas as pd
import pathlib

# Define the database file in the current root project directory
db_file = pathlib.Path("project.sqlite3")

def create_database():
    """Function to create a database. Connecting for the first time
    will create a new database file if it doesn't exist yet.
    Close the connection after creating the database
    to avoid locking the file."""
    try:
        conn = sqlite3.connect(db_file)
        conn.close()
        print("Database created successfully.")
    except sqlite3.Error as e:
        print("Error creating the database:", e)

def main():
    create_database()

if __name__ == "__main__":
    main()

Database created successfully.


2. Table Creation

In [34]:
# Start by deleting any tables if they exist already
# We want to be able to re-run this script as needed.
# DROP tables in reverse order of creation 
# DROP dependent tables (with foreign keys) first
# Makes the script idempotent (rerunnable) using the most current statements each time we run

sql_script = """
DROP TABLE IF EXISTS books;
DROP TABLE IF EXISTS authors;

CREATE TABLE authors (
    author_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    year_born INTEGER
);

CREATE TABLE books (
    book_id TEXT PRIMARY KEY,
    title TEXT,
    year_published INTEGER,
    author_id TEXT,
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
);
"""

# Connect to the database and execute the SQL script
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.executescript(sql_script)
conn.commit()
conn.close()
print("Tables created successfully.")

Tables created successfully.


Creating reusable function to execute a SQL command from a file. Just provide the database and a path to the sql with the command.

In [35]:
def create_tables():
    """Function to read and execute SQL statements to create tables"""
    try:
        with sqlite3.connect(db_file) as conn:
            sql_file = pathlib.Path("sql", "create_tables.sql")
            with open(sql_file, "r") as file:
                sql_script = file.read()
            conn.executescript(sql_script)
            print("Tables created successfully.")
    except sqlite3.Error as e:
        print("Error creating tables:", e)

Creating a function for each task keeps our code organized. Update the main () function to call this one as well.

In [36]:
def main():
    create_database()
    create_tables()

3a. Creating records with INSERT INTO
We can populate our tables in several ways - we'll look at two: (1) Using SQL INSERT INTO statements and (2) populating tables directly from compatible data files.

These simple examples use INSERT INTO statements to populate tables. These statements make it easy to create and populate your database when using online SQL tools for practice. Here are some simple examples (using a slightly different author schema):

In [37]:
# ...existing code...
import sqlite3
db_file = "project.sqlite3"

with sqlite3.connect(db_file) as conn:
    cur = conn.cursor()

    # single insert (parameterized)
    cur.execute(
        "INSERT INTO authors (author_id, first_name, last_name, year_born) VALUES (?, ?, ?, ?)",
        ("A1", "John", "Doe", 1970)
    )
    print("cursor.lastrowid:", cur.lastrowid)

    # multiple inserts (executemany)
    cur.executemany(
        "INSERT INTO authors (author_id, first_name, last_name, year_born) VALUES (?, ?, ?, ?)",
        [
            ("A2", "Alice", "Smith", 1980),
            ("A3", "Bob", "Johnson", 1990),
            ("A4", "Eva", "Brown", 1985)
        ]
    )

    # insert then fetch SQLite's last_insert_rowid() from same connection
    cur.execute(
        "INSERT INTO authors (author_id, first_name, last_name, year_born) VALUES (?, ?, ?, ?)",
        ("A5", "Jane", "Green", 1975)
    )
    cur.execute("SELECT last_insert_rowid()")
    print("SELECT last_insert_rowid():", cur.fetchone()[0])

    conn.commit()
# ...existing code...

cursor.lastrowid: 1
SELECT last_insert_rowid(): 5


Populating our two project tables 
sql/insert_records.sql

In [38]:
# Insert authors and books data using Python and sqlite3

sql_insert = """
INSERT INTO authors (author_id, first_name, last_name, year_born)
VALUES
    ('10f88232-1ae7-4d88-a6a2-dfcebb22a596', 'Harper', 'Lee', NULL),
    ('c3a47e85-2a6b-4196-a7a8-8b55d8fc1f70', 'George', 'Orwell', NULL),
    ('e0b75863-866d-4db4-85c7-df9bb8ee6dab', 'F. Scott', 'Fitzgerald', NULL),
    ('7b144e32-7ff4-4b58-8eb0-e63d3c9f9b8d', 'Aldous', 'Huxley', NULL),
    ('8d8107b6-1f24-481c-8a21-7d72b13b59b5', 'J.D.', 'Salinger', NULL),
    ('0cc3c8e4-e0c0-482f-b2f7-af87330de214', 'Ray', 'Bradbury', NULL),
    ('4dca0632-2c53-490c-99d5-4f6d41e56c0e', 'Jane', 'Austen', NULL),
    ('16f3e0a1-24cb-4ed6-a50d-509f63e367f7', 'J.R.R.', 'Tolkien', NULL),
    ('06cf58ab-90f1-448d-8e54-055e4393e75c', 'J.R.R.', 'Tolkien', NULL),
    ('6b693b96-394a-4a1d-a4e2-792a47d7a568', 'J.K.', 'Rowling', NULL);

INSERT INTO books (book_id, title, year_published, author_id)
VALUES
    ('d6f83870-ff21-4a5d-90ab-26a49ab6ed12', 'To Kill a Mockingbird', 1960, '10f88232-1ae7-4d88-a6a2-dfcebb22a596'),
    ('0f5f44f7-44d8-4f49-b8c4-c64d847587d3', '1984', 1949, 'c3a47e85-2a6b-4196-a7a8-8b55d8fc1f70'),
    ('f9d9e7de-c44d-4d1d-b3ab-59343bf32bc2', 'The Great Gatsby', 1925, 'e0b75863-866d-4db4-85c7-df9bb8ee6dab'),
    ('38e530f1-228f-4d6e-a587-2ed4d6c44e9c', 'Brave New World', 1932, '7b144e32-7ff4-4b58-8eb0-e63d3c9f9b8d'),
    ('c2a62a4b-cf5c-4246-9bf7-b2601d542e6d', 'The Catcher in the Rye', 1951, '8d8107b6-1f24-481c-8a21-7d72b13b59b5'),
    ('3a1d835c-1e15-4a48-8e8c-b12239604e98', 'Fahrenheit 451', 1953, '0cc3c8e4-e0c0-482f-b2f7-af87330de214'),
    ('c6e67918-e509-4a6b-bc3a-979f6ad802f0', 'Pride and Prejudice', 1813, '4dca0632-2c53-490c-99d5-4f6d41e56c0e'),
    ('be951205-6cc2-4b3d-96f1-7257b8fc8c0f', 'The Hobbit', 1937, '16f3e0a1-24cb-4ed6-a50d-509f63e367f7');
"""

# Create a new connection and cursor for this operation
with sqlite3.connect(db_file) as conn:
    cursor = conn.cursor()
    cursor.executescript(sql_insert)
    conn.commit()
    print("Authors and books inserted successfully.")


Authors and books inserted successfully.


3b. Creating Records from Data Files
Populating tables right from data at rest (static data files, e.g., csv files) is also very common. For this project, we have our csv files in the data folder. We can write a function to populate them right from the CSV using pandas.  Note that the data must be compatible with our table structure for the direct import to work. Typically, there will be much more data cleaning and munging required to go from one system's Excel or csv files into a centralized relational database system. Optional/aside: For modern data storage formats, look up data warehouses, data lakes, and data lakehouses. 

In [39]:
def insert_data_from_csv():
    """Function to use pandas to read data from CSV files (in 'data' folder)
    and insert the records into their respective tables."""
    try:
        author_data_path = pathlib.Path("data", "authors.csv")
        book_data_path = pathlib.Path("data", "books.csv")
        authors_df = pd.read_csv(author_data_path)
        books_df = pd.read_csv(book_data_path)
        with sqlite3.connect(db_file) as conn:
            # use the pandas DataFrame to_sql() method to insert data
            # pass in the table name and the connection
            authors_df.to_sql("authors", conn, if_exists="replace", index=False)
            books_df.to_sql("books", conn, if_exists="replace", index=False)
            print("Data inserted successfully.")
    except (sqlite3.Error, pd.errors.EmptyDataError, FileNotFoundError) as e:
        print("Error inserting data:", e)

After adding your function to either insert records from csv or insert records by running a sql script, update your main() function to call your new function that inserts records. For example:

In [40]:
def main():
    create_database()
    create_tables()
    insert_data_from_csv()

4. Reading Records with SQL SELECT
   SQL makes it easy to retrieve information from tables as well. Understand how to use SELECT queries to get different pieces of information, like listing all movies of a certain genre.